# Interactuando con SpreadSheets de google

Documentación del paquete [pygsheets](https://pygsheets.readthedocs.io/en/staging/)

## Descargo los ids de los cursos

In [34]:
import pygsheets

gc = pygsheets.authorize()

In [35]:
sh = gc.open('CURSOS-CONTENIDOS-REQUISITOS-OBJETIVOS')
hoja_web = sh.worksheet_by_title('Para web')

In [42]:
cursos_ids = hoja_web.range("G2:G122", returnas="range")

In [43]:
cursos_ids = [curso[0].value for curso in cursos_ids.cells]

In [44]:
cursos_ids

['79',
 '78',
 '479',
 '77',
 '38',
 '92',
 '85',
 '163',
 '57',
 '0',
 '919',
 '914',
 '61',
 '84',
 '0',
 '488',
 '329',
 '39',
 '109',
 '0',
 '168',
 '41',
 '42',
 '43',
 '955',
 '112',
 '4',
 '489',
 '476',
 '44',
 '6',
 '5',
 '90',
 '3',
 '45',
 '46',
 '47',
 '48',
 '49',
 '307',
 '374',
 '209',
 '167',
 '166',
 '254',
 '426',
 '165',
 '925',
 '86',
 '920',
 '40',
 '111',
 '253',
 '7',
 '99',
 '304',
 '252',
 '260',
 '52',
 '53',
 '55',
 '427',
 '923',
 '56',
 '164',
 '0',
 '54',
 '81',
 '160',
 '173',
 '159',
 '174',
 '171',
 '169',
 '172',
 '178',
 '179',
 '422',
 '421',
 '944',
 '946',
 '947',
 '948',
 '949',
 '950',
 '928',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '#N/A',
 '1094',
 '',
 '',
 '',
 '#N/A',
 '',
 '',
 '#N/A',
 '7']

## Obteniendo la imagen de cabecera de los cursos

In [16]:
import mechanicalsoup
from dotenv import load_dotenv
import os

load_dotenv()

ADMIN_USER = os.getenv("AULARAGON_ADMIN_USER")
PASSWORD = os.getenv("AULARAGON_ADMIN_PASSWORD")

In [55]:
def get_ms_admin_browser():
    browser = mechanicalsoup.StatefulBrowser()
    browser.open("https://moodle.catedu.es/login/index.php")
    browser.select_form()
    browser["username"] = ADMIN_USER
    browser["password"] = PASSWORD
    browser.submit_selected()
    return browser

def get_course_image(browser, course_id):
    browser.open(f"https://moodle.catedu.es/course/view.php?id={course_id}")
    page = browser.get_current_page()
    course_image = page.find("div", class_="course-header-image")
    image_url = course_image.get('style').split('url("')[1].split('"); -webkit')[0]
    return course_id, image_url
    
no_image = ['', '#N/A', '0']
course_images = {}
browser = get_ms_admin_browser()
for curso in cursos_ids:
    if curso not in no_image:
        curso_id, curso_image = get_course_image(browser, curso)
        course_images[curso_id] = curso_image

In [57]:
len(course_images)

83

## Añadiendo la imagen de portada a la hoja de cálculo

In [65]:
cursos_imagenes_url= hoja_web.range("G2:H122", returnas="range")

In [66]:
cursos_imagenes_url[0][1].set_value(0)

<Cell H2 '0'>

In [ ]:
for row in cursos_imagenes_url:
    try:
        if row[0].value in list(course_images.keys()):
            row[1].set_value(course_images[row[0].value])
    except:
        print("no sé qué ha pasado")